## Variational AutoEncoder(VAE) 모델 설명 및 학습과 생성

In [ ]:
import os
from easydict import EasyDict
from typing import List, Tuple, Dict, Union

from laiddmg import (
  VAEConfig,
  Tokenizer,
  VAEModel,
  get_rawdataset,
  get_dataset,
)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim

## configuration, tokenizer, model 생성

* `VAEConfig` class:
  * 모델을 구성하기 위해 필요한 정보(`hidden_dim`, `num_layers` 등)들이 담긴 class입니다.
  * 자세한 코드는 [`laiddmg/models/vae/configuration.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/configuration.py)에 나와 있습니다.
* `Tokenizer` class:
  * `str`으로 된 SMILES 데이터를 미리 정의해둔 `vocab_dict`에 맞춰 token data(`int`)로 바꿔주는 역할을 합니다.
  * 자세한 코드는 [`laiddmg/tokenization_utils.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/tokenization_utils.py)에 나와 있습니다.
* `VAEModel` class:
  * 실제 모델을 만들어주는 클래스입니다.
  * `PyTorch`에서 제공하는 표준적인 방법으로 클래스를 구성하였습니다. tutorial은 [https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) 여기서 확인할 수 있습니다.
  * 이 모델은 Rafael Gómez-Bombarelli, et. al., [Automatic Chemical Design Using a Data-Driven Continuous Representation of Molecules](https://pubs.acs.org/doi/10.1021/acscentsci.7b00572)을 바탕으로 작성하였습니다.
  * 자세한 코드는 [`laiddmg/models/vae/modeling.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/modeling.py)에 나와 있습니다.

In [ ]:
model_type = 'vae'
config = VAEConfig()
tokenizer = Tokenizer()
model = VAEModel(config)

#### Print model configuration

In [ ]:
for k, v in config.__dict__.items():
  print(f'{k}: {v}')

#### How to use tokenizer

In [ ]:
tokenizer.vocab

In [ ]:
smiles = 'c1ccccc1'  # 벤젠
tokenizer(smiles)

#### Print model's informations

In [ ]:
model

In [ ]:
print(f'model type: {config.model_type}')
print(f'model device: {model.device}')
print(f'model dtype: {model.dtype}')
print(f'number of training parameters: {model.num_parameters()}')

## Model 체크

* model의 input으로는 `input_ids`와 `lengths`가 필요합니다. 
* `input_ids`는 `tokenizer`를 통해 SMILES character를 각각 token number로 바꾼 결과입니다.
* `lengths`는 각 문장(각 SMILES 데이터)의 sequence length 정보입니다.

In [ ]:
smiles = 'c1ccccc1'
inputs = tokenizer(smiles)
inputs

In [ ]:
# outputs, z_mu, z_logvar = model(**inputs)
outputs, z_mu, z_logvar = model(input_ids=inputs['input_ids'],
                                lengths=inputs['lengths'])

In [ ]:
print(f'outputs shape: {outputs.shape}')
print(f'latent vector mean shape: {z_mu.shape}')
print(f'latent vector log variance shape: {z_logvar.shape}')

### VAE architecture

<img width="1183" alt="vae-gaussian" src="https://user-images.githubusercontent.com/11681225/131422864-d5b168ba-ae6a-4bb0-bccc-c985a7b95e00.png">

### Encoder 살펴보기

* 자세한 코드는 [`laiddmg/models/vae/modeling.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/modeling.py)에 나와 있습니다.

In [ ]:
class Encoder(nn.Module):

  def __init__(self, config: VAEConfig, embeddings: nn.Module = None):
    super(Encoder, self).__init__()
    self.vocab_size = config.vocab_size
    self.embedding_dim = config.embedding_dim
    self.encoder_hidden_dim = config.encoder_hidden_dim
    self.encoder_num_layers = config.encoder_num_layers
    self.encoder_dropout = config.encoder_dropout
    self.latent_dim = config.latent_dim
    self.padding_value = config.padding_value

    if embeddings is not None:
      self.embeddings = embeddings
    else:
      self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim,
                                     padding_idx=self.padding_value)

    self.gru = nn.GRU(self.embedding_dim,
                      self.encoder_hidden_dim,
                      self.encoder_num_layers,
                      batch_first=True,
                      dropout=self.encoder_dropout if self.encoder_num_layers > 1 else 0)
    self.fc = nn.Linear(self.encoder_hidden_dim, self.latent_dim * 2)

  def forward(
    self,
    input_ids: torch.Tensor,  # (batch_size, seq_len)
    lengths: torch.Tensor,  # (batch_size,)
    **kwargs,
  ) -> Tuple[torch.Tensor]:
    x = self.embeddings(input_ids)  # x: (batch_size, seq_len, embedding_dim)
    x = rnn_utils.pack_padded_sequence(
      x,
      lengths.cpu(),
      batch_first=True,
      enforce_sorted=False,
    )
    _, hiddens = self.gru(x, None)  # hiddens: (num_layers, batch_size, encoder_hidden_dim)

    hiddens = hiddens[-1]  # hiddens: (batch_size, encoder_hidden_dim) for last layer

    z_mu, z_logvar = torch.split(self.fc(hiddens), self.latent_dim, dim=-1)
    # z_mu, z_logvar: (batch_size, latent_dim)

    return z_mu, z_logvar

In [ ]:
encoder = Encoder(config)
z_mu, z_logvar = encoder(**inputs)

In [ ]:
print(f'latent vector mean shape {z_mu.shape}')
print(f'latent vector log variance shape {z_logvar.shape}')

### Decoder 살펴보기

* 자세한 코드는 [`laiddmg/models/vae/modeling.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/modeling.py)에 나와 있습니다.

In [ ]:
class Decoder(nn.Module):

  def __init__(self, config: VAEConfig, embeddings: nn.Module = None):
    super(Decoder, self).__init__()
    self.vocab_size = config.vocab_size
    self.embedding_dim = config.embedding_dim
    self.latent_dim = config.latent_dim
    self.decoder_hidden_dim = config.decoder_hidden_dim
    self.decoder_num_layers = config.decoder_num_layers
    self.decoder_dropout = config.decoder_dropout
    self.input_dim = self.embedding_dim + self.latent_dim
    self.output_dim = config.vocab_size
    self.padding_value = config.padding_value

    if embeddings is not None:
      self.embeddings = embeddings
    else:
      self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim,
                                     padding_idx=self.padding_value)

    self.gru = nn.GRU(self.input_dim,
                      self.decoder_hidden_dim,
                      self.decoder_num_layers,
                      batch_first=True,
                      dropout=self.decoder_dropout if self.decoder_num_layers > 1 else 0)
    self.z2hidden = nn.Linear(self.latent_dim, self.decoder_hidden_dim)
    self.fc = nn.Linear(self.decoder_hidden_dim, self.output_dim)

  def forward(
    self,
    input_ids: torch.Tensor,  # (batch_size, seq_len)
    lengths: torch.Tensor,  # (batch_size,)
    z: torch.Tensor,  # (batch_size, latent_dim)
    **kwargs,
  ) -> Tuple[torch.Tensor]:
    x = self.embeddings(input_ids)  # x: (batch_size, seq_len, embedding_dim)
    hiddens = self.z2hidden(z)  # hiddens: (batch_size, decoder_hidden_dim)
    hiddens = hiddens.unsqueeze(0).repeat(self.decoder_num_layers, 1, 1)
    # hiddens: (num_layers, batch_size, decoder_hidden_dim)

    z_ = z.unsqueeze(1).repeat(1, x.shape[1], 1)  # z: (batch_size, seq_len, latent_dim)
    x = torch.cat((x, z_), dim=-1)  # x: (batch_size, seq_len, embedding_dim + latent_dim)

    x = rnn_utils.pack_padded_sequence(
      x,
      lengths.cpu(),
      batch_first=True,
      enforce_sorted=False
    )
    x, _ = self.gru(x, hiddens)
    x, _ = rnn_utils.pad_packed_sequence(
      x,
      batch_first=True,
    )  # x: (batch_size, seq_len, hidden_dim)
    outputs = self.fc(x)  # outputs: (batch_size, seq_len, vocab_size)

    return outputs

In [ ]:
decoder = Decoder(config)
outputs = decoder(**inputs, z=torch.randn(1, 128))

In [ ]:
print(f'decoder outputs shape: {outputs.shape}')
print(f'input_ids shape: {inputs["input_ids"].shape}')

### VAE model class

* 자세한 코드는 [`laiddmg/models/vae/modeling.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/modeling.py)에 나와 있습니다.


### VAE model step by step으로 알아보기

1. 인풋 데이터(`input_ids`)를 Encoder(`encoder`)에 넣는다.
2. `z_mu`와 `z_logvar`값에 reparametrization trick을 적용하여 실제 sample된 latent vector `z`를 만든다.
3. 인코딩된 정보 latent vector `z`와 인풋 데이터(`encoder`의 인풋과 같다)를 이용하여 Decoder(`decoder`)에 적용시킨다.

In [ ]:
class _VAEModel(nn.Module):

  def __init__(self, config: VAEConfig):
    super(VAEModel, self).__init__()
    self.config = config
    self.vocab_size = config.vocab_size
    self.embedding_dim = config.embedding_dim
    self.latent_dim = config.latent_dim
    self.padding_value = config.padding_value

    self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim,
                                   padding_idx=self.padding_value)

    self.encoder = Encoder(self.config, self.embeddings)
    self.decoder = Decoder(self.config, self.embeddings)

  def reparameterize(self, mean, logvar):
    epsilon = torch.rand_like(mean)
    z = epsilon * torch.exp(logvar * .5) + mean  # mean, logvar, z: (batch_size, latent_dim)

    return z

  def forward(
    self,
    input_ids: torch.Tensor,  # (batch_size, seq_len)
    lengths: torch.Tensor,  # (batch_size,)
    **kwargs,
  ) -> Tuple[torch.Tensor]:
    z_mu, z_logvar = self.encoder(input_ids, lengths)
    z = self.reparameterize(z_mu, z_logvar)  # z: (batch_size, latent_dim)
    y = self.decoder(input_ids, lengths, z)  # y: (batch_size, seq_len, vocab_size)

    return y, z_mu, z_logvar

#### 1. 인풋 데이터(`input_ids`)를 Encoder(`encoder`)에 넣는다.

In [ ]:
z_mu, z_logvar = model.encoder(**inputs)

In [ ]:
print(f'latent vector mean shape: {z_mu.shape}')
print(f'latent vector log variance shape: {z_logvar.shape}')

#### 2. `z_mu`와 `z_logvar`값에 reparametrization trick을 적용하여 실제 sample된 latent vector `z`를 만든다.

In [ ]:
epsilon = torch.rand_like(z_mu)
z = epsilon * torch.exp(z_logvar * .5) + z_mu

In [ ]:
print(f'latent vector shape: {z.shape}')

#### 3. 인코딩된 정보 latent vector `z`와 인풋 데이터(`encoder의 인풋과 같다)를 이용하여 decoder에 적용시킨다.

In [ ]:
outputs = model.decoder(**inputs, z=z)

In [ ]:
print(f'outputs shape: {outputs.shape}')

## Data 얻기

* [Molecular Sets (MOSES): A benchmarking platform for molecular generation models](https://github.com/molecularsets/moses)에서 사용한 ZINC데이터를 random sampling을 통해 `train : test = 250000 : 30000`으로 나누었습니다.
* 실제 데이터 파일 경로는 아래와 같습니다.
  * [`datasets/moses`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/datasets/moses)
* `get_rawdataset`함수를 이용하여 얻은 데이터는 각 항목이 SMILES `str`데이터로 이루어진 `np.ndarray`입니다.
* 이 rawdataset을 사용하기 편하게 `custom Dataset` class를 만들었습니다.
  * `custom Dataset`을 만드는 간단한 예제는 [PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files)에 있습니다.
  * 자세한 코드는 [`laiddmg/ᅟdatasets.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/datasets.py)에 나와 있습니다.

In [ ]:
train = get_rawdataset('train')

In [ ]:
train

In [ ]:
print(f'number of training dataset: {len(train)}')
print(f'raw data type: {type(train[0])}')

#### `model`에 sample data 적용해보기

In [ ]:
sampled_data = train[:4]
inputs = tokenizer(sampled_data)
inputs

#### `inputs`를 `model`의 입력값으로 넣기

In [ ]:
outputs, z_mu, z_logvar = model(**inputs)

In [ ]:
print(f'outputs shape: {outputs.shape}')
print(f'latent vector mean shape: {z_mu.shape}')
print(f'latent vector log variance shape: {z_logvar.shape}')

### PyTorch `Dataset`, `DataLoader` 얻기

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_dataset = get_dataset(train, tokenizer)

In [ ]:
train_dataset[1000]

#### `input_id`와 `target`의 관계

RNN을 이용한 생성모델(generative model)은 language model의 학습방법을 이용한다.
Language model은 간단하게 이야기하면 다음 단어를 예측하는 모델이다.
다음 단어를 예측한다는 뜻은 RNN 그림을 보면 쉽게 이해할 수 있다.

![RNN-input-target](https://user-images.githubusercontent.com/11681225/129859647-af31934a-0eea-4ad8-9a85-2d3c2a75f517.jpeg)

위와 같이 input data의 token이 하나씩 이동한 것이 target이 되는 것이다.

In [ ]:
def _pad_sequence(data: List[torch.Tensor],
                  padding_value: int = 0) -> torch.Tensor:
  return rnn_utils.pad_sequence(data,
                                batch_first=True,
                                padding_value=padding_value)

In [ ]:
def _collate_fn(batch: List[Dict[str, Union[torch.Tensor, str, int]]],
                **kwargs) -> Dict[str, Union[torch.Tensor, List[str], List[int]]]:

  indexes = [item['index'] for item in batch]
  smiles = [item['smiles'] for item in batch]
  input_ids = [item['input_id'] for item in batch]
  targets = [item['target'] for item in batch]
  lengths = [item['length'] for item in batch]

  padding_value = tokenizer.padding_value
  input_ids = _pad_sequence(input_ids, padding_value)
  targets = _pad_sequence(targets, padding_value)
  lengths = torch.LongTensor(lengths)

  return {'input_ids': input_ids,
          'targets': targets,
          'lengths': lengths,
          'smiles': smiles,
          'indexes': indexes}

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=4,
                              shuffle=True,
                              collate_fn=_collate_fn,
                             )

### Train without `Trainer` class

* 실제 사용할 수 있게 패키징한 코드에서는 `Trainer` class를 만들어 사용하기 편리하게 모듈화 시켰습니다.
* 하지만 해당 Jupyter notebook은 이해를 돕기위해 모듈화 되어 있는 코드를 풀어서 블록 단위로 나타내었습니다.
* `Trainer`에 관련된 자세한 코드는 아래 링크에 있습니다.
  * [`laiddmg/trainer.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/trainer.py)
  * [`laiddmg/models/vae/vae_trainer.py`](https://github.com/ilguyi/LAIDD-molecular-generation/blob/main/laiddmg/models/vae/vae_trainer.py)

#### loss function and optimizer 생성

In [ ]:
training_args = EasyDict({
  'output_dir': 'outputs/vae/jupyter1',
  'num_train_epochs': 10,
  'batch_size': 256,
  'lr': 1e-3,
})

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=training_args.batch_size,
                              shuffle=True,
                              collate_fn=_collate_fn,
                             )

In [ ]:
reconstruction_loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.padding_value)
optimizer = optim.Adam(model.parameters(), lr=training_args.lr)

### Training

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model = model.to(device)
print(model.device)

### KL annealing

In [ ]:
from utils import AnnealingSchedules
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
kl_annealing = AnnealingSchedules(
  method='cycle_sigmoid',  # cycle_linear, cycle_sigmoid, cycle_cosine
  update_unit='step',  # epoch, step
  num_training_steps=100,
  num_training_steps_per_epoch=10,
  start_weight=0.0,
  stop_weight=1.0,
)

In [ ]:
kl_annealing_weight = []
for step in range(100):
  kl_annealing_weight.append(kl_annealing(step)) 

In [ ]:
plt.plot(kl_annealing_weight)

### Training

In [ ]:
def save_model(epoch: int, global_step: int, model: nn.Module):
  checkpoint_dir = os.path.join(training_args.output_dir)
  os.makedirs(checkpoint_dir, exist_ok=True)
  ckpt_name = f'ckpt_{epoch:03d}.pt'
  ckpt_path = os.path.join(checkpoint_dir, ckpt_name)
  
  torch.save({'epoch': epoch,
              'global_step': global_step,
              'model_state_dict': model.state_dict()},
             ckpt_path)
  print(f'saved {model.config.model_type} model at epoch {epoch}.')

### KL Divergence Loss

![vae kl loss](https://user-images.githubusercontent.com/11681225/131319712-3ca94a3c-0f72-4b9b-9a37-1b91e53c608c.jpeg)

In [ ]:
model.train()
global_step = 0

kl_annealing = AnnealingSchedules(
  method='cycle_linear',  # cycle_linear, cycle_sigmoid, cycle_cosine
  update_unit='epoch',  # epoch, step
  num_training_steps=len(train_dataloader) * training_args.num_train_epochs,
  num_training_steps_per_epoch=len(train_dataloader),
  start_weight=0.0,
  stop_weight=0.05,
)

for epoch in range(1, training_args.num_train_epochs + 1):
  print(f'\nStart training: {epoch} Epoch\n')
  
  for i, data in enumerate(train_dataloader, 1):
    optimizer.zero_grad()
    
    data['input_ids'] = data['input_ids'].to(device)
    data['targets'] = data['targets'].to(device)
    outputs, z_mu, z_logvar = model(data['input_ids'], data['lengths'])
    
    reconstruction_loss = reconstruction_loss_fn(outputs.view(-1, outputs.shape[-1]),
                                                 data['targets'].view(-1))
    
    kl_loss = .5 * (torch.exp(z_logvar) + z_mu**2 - 1. - z_logvar).sum(1).mean()
    
    kl_annealing_weight = kl_annealing(global_step)
    
    total_loss = reconstruction_loss + kl_annealing_weight * kl_loss
    
    total_loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(),
                             max_norm=50)
    optimizer.step()
    global_step += 1
    
    if global_step % 100 == 0:
      print(f'{epoch} Epochs | {i}/{len(train_dataloader)} | reconst_loss: {reconstruction_loss.item():.4g} | '
            f'kl_loss: {kl_loss:.4g}, total_loss: {total_loss:.4g}, '
            f'kl_annealing: {kl_annealing(global_step - 1):.4g} ')

  save_model(epoch, global_step, model)
  
print('Training done!!')

## Generate new SMILES

* model을 학습한 후에는 학습된 모델을 `load`하여 SMILES를 생성할 준비를 합니다.
* `model.generate`함수를 이용하면 새로운 SMILES sequence를 만들수 있습니다.
* 여기서는 generation의 각 과정을 하나씩 설명합니다.
* 자세한 코드는 [`laiddmg/generate.py`](https://github.com/ilguyi/LAIDD-molecule-generation/blob/main/laiddmg/generate.py)에 나와 있습니다.

In [ ]:
checkpoint_dir = training_args.output_dir
model = VAEModel.from_pretrained(config, os.path.join(f'{checkpoint_dir}',
                                                      f'ckpt_{training_args.num_train_epochs:03d}.pt'))
model.eval()

* 본 수업에서는 시간관계상 미리 학습한 `best_model`을 다운 받아서 씁니다.

In [ ]:
!wget 'https://www.dropbox.com/s/751pqnlgwqnqkby/vae_best.tar.gz?dl=0'
!tar xvzf vae_best.tar.gz?dl=0
!rm -f vae_best.tar.gz?dl=0
!mv best_model/ outputs/vae/

In [ ]:
model = VAEModel.from_pretrained(config,
                                 os.path.join('./outputs/vae/best_model/best_model.pt'))
model = model.to(device)
model.eval()

In [ ]:
batch_size_for_generate = 4

In [ ]:
outputs = model.generate(tokenizer=tokenizer,
                         max_length=128,
                         num_return_sequences=batch_size_for_generate,
                         skip_special_tokens=True)

In [ ]:
outputs

### generation 과정 step by step으로 알아보기

* step 1. `input_ids`변수에 첫 번째 token 데이터인 `<START> token` 넣기
* step 2. prior distribution(Guassian distribution)에서 latent vector `z` 샘플링
* step 3. latent vector `z`를 decoder의 initial state로 넣기 위해 Linear레이어 적용
* step 4. `input_ids`데이터를 `embedding`에 넣어 embedded input 얻기
* step 5. latent vector `z`를 (모든) input data에 concatenate
* step 6. concatenate한 input 데이터 gru에 넣기
* step 7. `outputs`을 `Linear`레이어를 통과시켜서 `next_token_logits`을 얻기
* step 8. `next_token_logits`을 `softmax`를 통해 확률분포를 얻음
* step 9. 이 확률분포를 기반한 sampling 작업을 함 (`torch.multinomial`을 이용)
* step 10. 실제로 sampling된 값이 `next_tokens`이 되고 이게 다음 스텝의 rnn 인풋으로 쓰임 (`input_ids = next_tokens`)
* step 11. step 2 ~ step 10과정을 반복

In [ ]:
model = model.to(device)

#### step 1. `input_ids`변수에 `<START> token` 넣기

In [ ]:
initial_inputs = torch.full((batch_size_for_generate, 1),
                            tokenizer.convert_token_to_id(tokenizer.start_token),
                            dtype=torch.long,
                            device=model.device)
generated_sequences = initial_inputs
input_ids = initial_inputs

In [ ]:
input_ids

#### step 2. prior distribution(Guassian distribution)에서 latent vector `z` 샘플링

In [ ]:
z = model.sample_gaussian_dist(batch_size_for_generate)  # z: [batch_size, latent_dim]
z_ = z.unsqueeze(1)  # z_: [batch_size, 1, latent_dim]
# z_: step 5에서 input과 concatenate 하기 위해 shape을 맞춰줌

In [ ]:
print(z.shape)
print(z_.shape)

#### step 3. latent vector `z`를 decoder의 initial state로 넣기 위해 Linear레이어 적용

In [ ]:
hiddens = model.decoder.z2hidden(z)  # hiddens: [batch_size, hidden_dim]
hiddens = hiddens.unsqueeze(0).repeat(model.config.decoder_num_layers, 1, 1)

In [ ]:
print(hiddens.shape)  # [decoder.num_layers, batch_size, hidden_dim]

#### step 4. `input_ids`데이터를 `embedding`에 넣어 embedded input 얻기

In [ ]:
x = model.embeddings(input_ids)  # x: [batch_size, 1, embedding_dim]

In [ ]:
x.shape

#### step 5. latent vector `z`를 (모든) input data에 concatenate

* 매 token 마다 latent vector의 정보를 추가하여 성능을 높이기 위해서

In [ ]:
x = torch.cat((x, z_), dim=-1)  # x: [batch_size, 1, embedding_dim + latent_dim]

#### step 6. concatenate한 input 데이터 gru에 넣기

In [ ]:
x, hiddens = model.decoder.gru(x, hiddens)

#### step 7. `outputs`을 `Linear`레이어를 통과시켜서 `next_token_logits`을 얻기

In [ ]:
logits = model.decoder.fc(x)
next_token_logits = logits.squeeze(1)

In [ ]:
logits.shape

In [ ]:
next_token_logits.shape

#### step 8. `next_token_logits`을 `softmax`를 통해 확률분포를 얻음

In [ ]:
probabilities = F.softmax(next_token_logits, dim=-1)

In [ ]:
probabilities[0]

#### step 9. 이 확률분포를 기반한 sampling 작업을 함 ([`torch.multinomial`](https://pytorch.org/docs/stable/generated/torch.multinomial.html?highlight=multinomial#torch.multinomial)을 이용)

In [ ]:
next_tokens = torch.multinomial(probabilities, num_samples=1)
next_tokens

참고 `tokenizer.vocab`

```python
{('#', 4), ('(', 5), (')', 6), ('-', 7),
 ('1', 8), ('2', 9), ('3', 10), ('4', 11), ('5', 12), ('6', 13), ('=', 14),
 ('B', 15), ('C', 16), ('F', 17), ('H', 18), ('N', 19), ('O', 20), ('S', 21),
 ('[', 22), (']', 23), ('c', 24), ('l', 25), ('n', 26), ('o', 27), ('r', 28), ('s', 29)}
```

#### step 10. 실제로 sampling된 값이 `next_tokens`이 되고 이게 다음 스텝의 rnn 인풋으로 쓰임 (`input_ids = next_tokens`)

In [ ]:
inputs_ids = next_tokens
generated_sequences = torch.cat((generated_sequences, next_tokens), dim=1)
generated_sequences

#### 위의 과정을 모듈화해서 `generate`함수를 만들었습니다.

In [ ]:
outputs = model.generate(tokenizer=tokenizer,
                         max_length=128,
                         #num_return_sequences=batch_size_for_generate,
                         num_return_sequences=256,
                         skip_special_tokens=True)

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
mols = []
for s in outputs:
  try:
    mol = Chem.MolFromSmiles(s)
  except:
    pass
  if mol is not None:
    mols.append(mol)

In [ ]:
len(mols)

In [ ]:
mols[0]

In [ ]:
mols[1]

In [ ]:
mols[2]